In [9]:
import pandas as pd
import numpy as np
import random

In [10]:

# Carregar os ficheiros
voos = pd.read_csv("flights_com_modelos.csv")
aeroportos = pd.read_csv("aeroportos_formatado.csv")
aeronaves = pd.read_csv("aeronaves.csv")
companhias = pd.read_csv("airlines.csv")

# 2. Renomear colunas relevantes
aeronaves.rename(columns={"Model": "Modelo Avião", "Company": "Fabricante"}, inplace=True)
companhias.rename(columns={"Name": "nome_companhia_aerea", "IATA": "AIRLINE"}, inplace=True)

# 3. Juntar os dados
voos = voos.merge(aeronaves, on="Modelo Avião", how="left")
voos = voos.merge(companhias[["AIRLINE", "nome_companhia_aerea"]], on="AIRLINE", how="left")
voos = voos.merge(aeroportos[["Codigo_IATA", "Nome"]], left_on="ORIGIN", right_on="Codigo_IATA", how="left")
voos.rename(columns={"Nome": "aeroporto_origem"}, inplace=True)
voos = voos.merge(aeroportos[["Codigo_IATA", "Nome"]], left_on="DEST", right_on="Codigo_IATA", how="left")
voos.rename(columns={"Nome": "aeroporto_destino"}, inplace=True)

# 4. Preparar dados finais
voos_final = pd.DataFrame({
    "nr_voo": voos["FL_NUMBER"],
    "data_partida": voos["FL_DATE"],
    "nome_companhia_aerea": voos["nome_companhia_aerea"],
    "aeroporto_origem": voos["aeroporto_origem"],
    "aeroporto_destino": voos["aeroporto_destino"],
    "hora_partida_real": voos["DEP_TIME"],
    "hora_partida_esperada": voos["CRS_DEP_TIME"],
    "hora_chegada_real": voos["ARR_TIME"],
    "hora_chegada_esperada": voos["CRS_ARR_TIME"],
    "cancelado": voos["CANCELLED"],
    "desviado": voos["DIVERTED"],
    "distancia": voos["DISTANCE"],
    "atraso_companhia_aerea": voos["DELAY_DUE_CARRIER"],
    "atraso_metereologia": voos["DELAY_DUE_WEATHER"],
    "atraso_SNA": voos["DELAY_DUE_NAS"],
    "atraso_seguranca": voos["DELAY_DUE_SECURITY"],
    "atraso_voo_anterior": voos["DELAY_DUE_LATE_AIRCRAFT"],
    "Modelo_Avião": voos["Modelo Avião"],
    "Fabricante": voos["Fabricante"],
    "avaliacao": voos["Rating"]
})


In [11]:
# 5. Gerar campos climáticos com base nos atrasos
def simular_clima(row):
    clima = {
        "temperatura": np.random.uniform(15, 25),
        "precipitacao": np.random.uniform(0, 5),
        "visibilidade": np.random.uniform(8, 15),
        "cobertura_nuvens": np.random.uniform(0, 50),
        "velocidade": np.random.uniform(5, 20),
        "direcao": np.random.uniform(0, 360)
    }

    if row["atraso_metereologia"] > 0:
        clima["temperatura"] = np.random.uniform(25, 40)
        clima["precipitacao"] = np.random.uniform(5, 50)
        clima["cobertura_nuvens"] = np.random.uniform(60, 100)

    if row["atraso_companhia_aerea"] > 0:
        clima["precipitacao"] += np.random.uniform(3, 20)

    if row["atraso_SNA"] > 0:
        clima["velocidade"] = np.random.uniform(20, 60)
        clima["direcao"] = np.random.uniform(0, 360)

    if row["atraso_seguranca"] > 0:
        clima["cobertura_nuvens"] = max(clima["cobertura_nuvens"], np.random.uniform(70, 100))
        clima["visibilidade"] = np.random.uniform(0.5, 5)

    if row["atraso_voo_anterior"] > 0:
        for key in clima:
            clima[key] *= np.random.uniform(1.05, 1.15)

    return pd.Series(clima)

# 6. Aplicar simulação
clima_df = voos_final.apply(simular_clima, axis=1)
voos_completos = pd.concat([voos_final, clima_df], axis=1)

# 7. Exportar para CSV
voos_completos.to_csv("voos_completos_clima.csv", index=False)

print("✅ Ficheiro 'voos_completos_clima.csv' criado com sucesso.")

✅ Ficheiro 'voos_completos_clima.csv' criado com sucesso.
